In [10]:
import tweepy
import json
import math
import glob
import csv
import zipfile
import zlib
import argparse
import os
import os.path as osp
import pandas as pd
from tweepy import TweepError
from time import sleep
from tqdm import tqdm

In [4]:
output_file = "/home/ubuntu/argminer/Experiments/HydratedTweets/000.csv_hydrated_tweets"
hydration_mode = "e"

output_file_noformat = output_file.split(".",maxsplit=1)[0]
print(output_file)
output_file = '{}'.format(output_file)
output_file_short = '{}_short.json'.format(output_file_noformat)

/home/ubuntu/argminer/Experiments/HydratedTweets/000.csv_hydrated_tweets


In [5]:
output_file_short

'/home/ubuntu/argminer/Experiments/HydratedTweets/000_short.json'

In [7]:
def is_retweet(entry):
    return 'retweeted_status' in entry.keys()

def get_source(entry):
    if '<' in entry["source"]:
        return entry["source"].split('>')[1].split('<')[0]
    else:
        return entry["source"]

TypeError: object of type '_io.TextIOWrapper' has no len()

In [11]:
print('creating minimized json master file')
with open(output_file_short, 'w') as outfile:
    with open(output_file) as json_data:
        for tweet in tqdm(json_data):
            data = json.loads(tweet) 
            if hydration_mode == "e":
                text = data["full_text"]
            else:
                text = data["text"]          
            t = {
                "created_at": data["created_at"],
                "text": text,
                "in_reply_to_screen_name": data["in_reply_to_screen_name"],
                "retweet_count": data["retweet_count"],
                "favorite_count": data["favorite_count"],
                "source": get_source(data),
                "id_str": data["id_str"],
                "is_retweet": is_retweet(data)
            }
            json.dump(t, outfile)
            outfile.write('\n')

creating minimized json master file


416118it [00:33, 12575.82it/s]


JSONDecodeError: Expecting value: line 1 column 1672 (char 1671)

In [12]:
f = csv.writer(open('{}.csv'.format(output_file_noformat), 'w'))
print('creating CSV version of minimized json master file') 
fields = ["favorite_count", "source", "text", "in_reply_to_screen_name", "is_retweet", "created_at", "retweet_count", "id_str"]                
f.writerow(fields)       
with open(output_file_short) as master_file:
    for tweet in master_file:
        data = json.loads(tweet)            
        f.writerow([data["favorite_count"], data["source"], data["text"].encode('utf-8'), data["in_reply_to_screen_name"], data["is_retweet"], data["created_at"], data["retweet_count"], data["id_str"].encode('utf-8')])


creating CSV version of minimized json master file


In [13]:
import pandas as pd

In [14]:
df = pd.read_csv("HydratedTweets/000.csv")

In [15]:
df.shape

(416106, 8)

In [28]:
df['text'] = df.text.apply(lambda x : eval(x).decode() if type(eval(x)) != str else eval(x))

,favorite_count,source,text,in_reply_to_screen_name,is_retweet,created_at,retweet_count,id_str
0,1,Twitter for iPhone,@SkyNewsBreak I had the coronavirus last year ...,SkyNewsBreak,False,Mon Jan 20 17:22:25 +0000 2020,0,b'1219309217015717888'
1,0,Twitter for iPhone,"@shehryar_taseer That’s 💯 true , \nCorona viru...",shehryar_taseer,False,Sat Jan 04 05:23:50 +0000 2020,0,b'1213330173736738817'
2,0,Twitter for Android,@tezuma75 Why #CCP keep on saying unknown caus...,tezuma75,False,Thu Jan 09 10:38:40 +0000 2020,0,b'1215221343479726081'
3,0,Twitter for Android,@lookner BBC has reported that coronavirus has...,lookner,False,Mon Jan 20 16:24:56 +0000 2020,0,b'1219294750844440576'
4,0,news2tw,Three U.S. Airports to Screen Passengers for C...,NaN,False,Fri Jan 17 19:08:02 +0000 2020,0,b'1218248632736067587'
...,...,...,...,...,...,...,...,...
416101,0,Twitter for iPad,#新冠肺炎\nA global emergency.\n※1/31/2020\n#加油武汉 ...,menghu_nankuru,False,Fri Jan 31 11:47:09 +0000 2020,0,b'1223211110821613570'
416102,0,Twitter for Android,"""2 confirmed Coronavirus case is being treated...",NaN,False,Fri Jan 31 11:46:34 +0000 2020,0,b'1223210963152834560'
416103,2,Twitter Web App,@DeonRDoiron1 Please watch 201 video \n\nUN pr...,DeonRDoiron1,False,Fri Jan 31 11:47:13 +0000 2020,1,b'1223211128198782977'
416104,1,Twitter Web App,@FOLLOW_DA_BUCKS @SchoonerLita @HometoBilo @Ju...,FOLLOW_DA_BUCKS,False,Fri Jan 31 11:46:43 +0000 2020,0,b'1223211001287299072'
